In [10]:
import pandas as pd
import numpy as np
import pickle

# Load data from previous notebook using pickle
with open('Pickle Files/step1.pkl', 'rb') as f:
    df = pickle.load(f)
    
print("Data loaded from Pickle Files/step1.pkl:", df.shape)

Data loaded from Pickle Files/step1.pkl: (10000, 9)


# Data Cleaning
To complete the cleaning process for e-commerce user event data, perform the following steps:

- Handle missing values (impute or drop as appropriate)

- Remove duplicate records (IQR, z-score)

- Detect and address outliers

- Validate data types and ranges

- Document any cleaning decisions


## Strategies for Handling Missing Values
After visualizing missing data, we can address it by either imputing (filling in) missing values or dropping rows/columns with missing data. The choice depends on the amount and importance of missing data:

- **Imputation:** Fill missing values using mean, median, mode, or other methods.
- **Dropping:** Remove rows or columns with excessive missing values if they are not critical for analysis.

Careful handling ensures data quality and reliable analysis.

In [11]:
# Handle missing values: drop rows with >50% missing, impute remaining with mean (numeric columns)
threshold = int(df.shape[1] * 0.5)
df = df.dropna(thresh=threshold)

# Impute remaining missing values in numeric columns with mean
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col] = df[col].fillna(df[col].mean())

print('Shape after cleaning:', df.shape)
df.isna().sum()

Shape after cleaning: (10000, 9)


event_time          0
event_type          0
product_id          0
category_id         0
category_code    3277
brand            1442
price               0
user_id             0
user_session        0
dtype: int64

In [12]:
# Remove duplicates
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"Removed {before - after} duplicate rows. Remaining rows: {after}")

Removed 3 duplicate rows. Remaining rows: 9997


## What are Outliers?

Outliers are data points that deviate significantly from other observations in your dataset.

They can occur due to:
- Data entry errors
- Measurement errors
- Natural variability (some users spend much more than others)

**Examples in your dataset:**
- Extremely high-priced items (price column)
- Unusually high number of purchases by a single user

**Why it matters for clustering:**
Clustering algorithms like K-Means are distance-based. Outliers can skew centroids, leading to poor cluster definitions.

In [13]:
# Detect and remove outliers using the IQR method for all numeric columns (collect indices and remove all at once)
import numpy as np
numeric_cols = df.select_dtypes(include=[np.number]).columns
outlier_index = set()
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_index.update(df[(df[col] < lower_bound) | (df[col] > upper_bound)].index)
print(f"Total unique outliers across all numeric columns: {len(outlier_index)}")
df = df.drop(outlier_index)
print(f"Shape after removing outliers: {df.shape}")

Total unique outliers across all numeric columns: 1461
Shape after removing outliers: (8536, 9)


## Validating Data Types and Value Ranges

After handling missing values, duplicates, and outliers, it's important to ensure that each column has the correct data type and that values fall within expected ranges. This step helps prevent errors in downstream analysis and modeling.
print("Data types before correction:")
print(df.dtypes)

for col in df.select_dtypes(include=[np.number]).columns:
    min_val = df[col].min()
    max_val = df[col].max()
    print(f"{col}: min={min_val}, max=`{max_val}")

print("\nData types after correction:")
print(df.dtypes)

In [14]:
# Validate and correct data types
if 'event_time' in df.columns:
    df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce')
if 'event_type' in df.columns:
    df['event_type'] = df['event_type'].astype(str)
if 'product_id' in df.columns:
    df['product_id'] = df['product_id'].astype(str)
if 'category_id' in df.columns:
    df['category_id'] = df['category_id'].astype('category')
if 'category_code' in df.columns:
    df['category_code'] = df['category_code'].astype(str)
if 'brand' in df.columns:
    df['brand'] = df['brand'].astype(str)
if 'price' in df.columns:
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
if 'user_id' in df.columns:
    df['user_id'] = df['user_id'].astype(str)
if 'user_session' in df.columns:
    df['user_session'] = df['user_session'].astype(str)

All Data Type Corrections Applied:-

The following columns were converted to their appropriate types for analysis:

- `event_time`: converted to datetime
- `event_type`: converted to string
- `product_id`: converted to string
- `category_id`: converted to category
- `category_code`: converted to string
- `brand`: converted to string
- `price`: converted to numeric
- `user_id`: converted to string
- `user_session`: converted to string

These conversions ensure that each column is interpreted correctly for downstream analysis and modeling.

In [15]:
# Save cleaned data for next notebook using pickle
import pickle

with open('Pickle Files/step2.pkl', 'wb') as f:
    pickle.dump(df, f)
    
print("Cleaned data saved as Pickle Files/step2.pkl:", df.shape)

Cleaned data saved as Pickle Files/step2.pkl: (8536, 9)
